### We will first try to train and optimize some ML algorithms to predict the survival of individuals, using all available and meaningful data, and then, following the data analysis, we search for the best algorithm only for predicting the survival of males, following the schema:
- if female:
    - if all family died:
        - predict die
    - else predict survive
- if male:
    - if all family survives:
        - predict survive
    - else predict survival using ML on data: IsYoung, PClass, and Embarked

In [143]:
using Pkg
Pkg.add(["CSV", "DataFrames", "Statistics", "MLJ", "MLJFlux", "Flux", "MLJLIBSVMInterface"])

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.6/Project.toml`
  No Changes to `~/.julia/environments/v1.6/Manifest.toml`


In [144]:
using CSV, DataFrames, Statistics, MLJ, MLJFlux, Flux, MLJLIBSVMInterface

### Read the data

In [92]:
data = CSV.File("data/train.csv") |> DataFrame
X_pred = CSV.File("data/test.csv") |> DataFrame;

### Let's create a pipeline for pre-processing the data
The columns we will use are:
* Sex
* IsYoung / CategoricalAge
* Pclass
* Embarked
* FamilySurvived

We will also have to change the scientific type of our data for it to work with the Julia ML algorithms.

In [63]:
# Function encodes the port name
function port_to_numerical(port)
    if ismissing(port) || port == "S"
        return 0 
    elseif port == "C"
        return 2
    else
        return 1
    end
end

port_to_numerical (generic function with 2 methods)

In [65]:
# regex to extract first name
family_name_regex = r"^([\w\-]+)"

# Function returns a dictionary containing family names as keys, 
# and an array with the "Survived" label for all family members as value 
function get_family_dict(data)
    family_dict = Dict()
    
    # extract string
    for value in eachrow(data)
        name = match(family_name_regex, value.Name).match
        if haskey(family_dict, name)
            # push survived value to set matching the name 
            push!(get(family_dict, name, nothing), value.Survived)
        else
            # if dict does not contain this family, create a new set containing the "Survived" value of this person
            family_dict[name] = [value.Survived]
        end
    end
    
    return family_dict
end

get_family_dict (generic function with 1 method)

In [66]:
# function returns 0 if all family died
# - 1 if the person is travelling alone or family has mixed survival
# - 2 if all family survived
function family_survived(example, family_dict)
    name = match(family_name_regex, example.Name).match
    
    if haskey(family_dict, name) && length(get(family_dict, name, nothing)) > 1
        value = get(family_dict, name, nothing)
        if mean(value) == 1
            # all family survived
            return 2
        elseif mean(value) == 0
            # all family died
            return 0
        end
        # mixed survival
        return 1
    else
        # person is travelling alone
        return 1
    end
end

family_survived (generic function with 1 method)

In [91]:
# Function for pre-processing the data
# If given training data, only the "data" parameter has to be speciffied.
# For data to be predicted, the family dictionary must also be passed.
# @return family_dict only for training data.
function pre_process(data; train=true, family_dict=missing)
    # Sex to numerical
    data.Sex = map(x -> x == "male" ? 0 : 1, data.Sex)
    
    # IsYoung / Age
    # TODO
    data.Age = map(x -> !ismissing(x) && x < 14 ? 1 : 0, data.Age)
    rename!(data, :Age => :IsYoung)
    
    # Embarked to numerical
    data.Embarked = map(x -> port_to_numerical(x), data.Embarked)
    
    # FamilySurvived
    if train
        family_dict = get_family_dict(data)
    end
    data.FamilySurvived = map(x -> family_survived(x, family_dict), eachrow(data))
        
    # drop cols
    select!(data, Not([:PassengerId, :Name, :SibSp, :Parch, :Fare, :Ticket, :Cabin]))
    
    # scitype
    coerce!(data, Count => Continuous)
    if train
        coerce!(data, :Survived => OrderedFactor)
        return family_dict
    end
end

pre_process (generic function with 3 methods)

In [93]:
family_dict = pre_process(data)
pre_process(X_pred, train=false, family_dict=family_dict);

### Our data now is in the right format. The next step is to break it into examples and labels.

In [94]:
y, X = unpack(data, ==(:Survived), x->true, rng=123);
# shuffle data with a seed to mentain a certain consistency between runs

In [97]:
train, test = partition(eachindex(y), 0.7, shuffle=true);

### Let's find models that can be fitted to our data

In [72]:
models(matching(X, y))

48-element Vector{NamedTuple{(:name, :package_name, :is_supervised, :docstring, :hyperparameter_ranges, :hyperparameter_types, :hyperparameters, :implemented_methods, :is_pure_julia, :is_wrapper, :load_path, :package_license, :package_url, :package_uuid, :prediction_type, :supports_online, :supports_weights, :input_scitype, :target_scitype, :output_scitype), T} where T<:Tuple}:
 (name = AdaBoostClassifier, package_name = ScikitLearn, ... )
 (name = AdaBoostStumpClassifier, package_name = DecisionTree, ... )
 (name = BaggingClassifier, package_name = ScikitLearn, ... )
 (name = BayesianLDA, package_name = MultivariateStats, ... )
 (name = BayesianLDA, package_name = ScikitLearn, ... )
 (name = BayesianQDA, package_name = ScikitLearn, ... )
 (name = BayesianSubspaceLDA, package_name = MultivariateStats, ... )
 (name = ConstantClassifier, package_name = MLJModels, ... )
 (name = DecisionTreeClassifier, package_name = BetaML, ... )
 (name = DecisionTreeClassifier, package_name = DecisionTr

In [110]:
function write_to_file(data, path::String)
    fw = open(path, "w")
    write(fw, "PassengerId,Survived\n")
    close(fw)
    fw = open(path, "a")
    index = 892
    for value in data
        write(fw, "$index,$value\n")
        index += 1
    end
    close(fw)
end

write_to_file (generic function with 1 method)

### Neural Networks time :D

### To get the best results, we must first normalize the data

In [73]:
stand = Standardizer()

Standardizer(
    features = Symbol[],
    ignore = false,
    ordered_factor = false,
    count = false) @372

In [103]:
Model = @load NeuralNetworkClassifier
# we'll use mini-batch for training the MLP for faster convergence
# also change the number of hidden neurons from the default 0 to 10 
#     to allow the network to learn the, possibly, more complex relations between features. 
model = Model(builder=MLJFlux.Short(n_hidden=10, σ=Flux.elu, dropout=0.2),
    epochs=200, batch_size=100,
    acceleration=CUDALibs());

import MLJFlux ✔


┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /home/ahautelman/.julia/packages/MLJModels/zYlo3/src/loading.jl:168
┌ Warning: `acceleration isa CUDALibs` but no CUDA device (GPU) currently live. 
└ @ MLJFlux /home/ahautelman/.julia/packages/MLJFlux/AeMUx/src/classifier.jl:39


In [104]:
pipe = @pipeline stand model;

In [105]:
# define some ranges for the hyper-parameters that we want to optimize.
r = range(pipe, :(neural_network_classifier.lambda), lower=0.0, upper=10.0)
r2 = range(pipe, :(neural_network_classifier.alpha), lower=0.0, upper=4.0)
r3 = range(pipe, :(neural_network_classifier.builder.dropout), lower=0, upper=0.6)

typename(MLJBase.NumericRange)(Float64, :(neural_network_classifier.builder.dropout), ... )

In [106]:
model = TunedModel(model=pipe,
                   ranges = [r, r2, r3],
                   resampling=Holdout(fraction_train=0.8, shuffle=true),
                   measures=cross_entropy,
                   repeats=5);

In [107]:
mach = machine(model, X, y)
MLJ.fit!(mach, rows=train)

┌ Info: Training Machine{ProbabilisticTunedModel{Grid,…},…} @865.
└ @ MLJBase /home/ahautelman/.julia/packages/MLJBase/KWyqX/src/machines.jl:342
┌ Info: Attempting to evaluate 1000 models.
└ @ MLJTuning /home/ahautelman/.julia/packages/MLJTuning/9sSuR/src/tuned_models.jl:564
Evaluating over 1000 metamodels: 100%[=========================] Time: 0:35:50
┌ Warning: `acceleration isa CUDALibs` but no CUDA device (GPU) currently live. 
└ @ MLJBase /home/ahautelman/.julia/packages/MLJBase/KWyqX/src/machines.jl:436


Machine{ProbabilisticTunedModel{Grid,…},…} @865 trained 1 time; caches data
  args: 
    1:	Source @800 ⏎ `Table{AbstractVector{Continuous}}`
    2:	Source @456 ⏎ `AbstractVector{OrderedFactor{2}}`


In [139]:
ŷ = predict(mach, X[test, :]);
misclassification_rate(mode.(ŷ), y[test])

0.14606741573033707

### Good score on the test set

In [138]:
prediction = convert(Array{Int64}, mode.(predict(mach, X_pred)));
write_to_file(prediction, "data/nn.csv")

### Kaggle score: 0.78708

In [141]:
MLJ.save("NN.jlso", mach)

### SVC algorithm

In [145]:
@load SVC
model = SVC()

import MLJLIBSVMInterface ✔


┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /home/ahautelman/.julia/packages/MLJModels/zYlo3/src/loading.jl:168


SVC(
    kernel = LIBSVM.Kernel.RadialBasis,
    gamma = 0.0,
    weights = nothing,
    cost = 1.0,
    cachesize = 200.0,
    degree = 3,
    coef0 = 0.0,
    tolerance = 0.001,
    shrinking = true,
    probability = false) @564

In [146]:
pipe = @pipeline stand model;

In [147]:
mach = machine(pipe, X, y)

Machine{Pipeline423,…} @698 trained 0 times; caches data
  args: 
    1:	Source @191 ⏎ `Table{AbstractVector{Continuous}}`
    2:	Source @996 ⏎ `AbstractVector{OrderedFactor{2}}`


In [148]:
fit!(mach, rows=train)

┌ Info: Training Machine{Pipeline423,…} @698.
└ @ MLJBase /home/ahautelman/.julia/packages/MLJBase/KWyqX/src/machines.jl:342
┌ Info: Training Machine{Standardizer,…} @476.
└ @ MLJBase /home/ahautelman/.julia/packages/MLJBase/KWyqX/src/machines.jl:342
┌ Info: Training Machine{SVC,…} @949.
└ @ MLJBase /home/ahautelman/.julia/packages/MLJBase/KWyqX/src/machines.jl:342


Machine{Pipeline423,…} @698 trained 1 time; caches data
  args: 
    1:	Source @191 ⏎ `Table{AbstractVector{Continuous}}`
    2:	Source @996 ⏎ `AbstractVector{OrderedFactor{2}}`


In [149]:
ŷ = predict(mach, X[test, :])
misclassification_rate(ŷ, y[test])

0.13108614232209737

### Even better score for the SVC

In [153]:
prediction = convert(Array{Int64}, predict(mach, X_pred))
write_to_file(prediction, "data/svc.csv")

### Kaggle score: 0.78468

In [154]:
MLJ.save("SVC.jlso", mach)